# Install and Import modules

In [1]:
# !tar xfvz /kaggle/input/ultralytics-for-offline-install/archive.tar.gz
# !pip install --no-index --find-links=./packages ultralytics
# !rm -rf ./packages

In [2]:
# !cp -r '/kaggle/input/hengck-czii-cryo-et-01/wheel_file' '/kaggle/working/'
# !pip install /kaggle/working/wheel_file/asciitree-0.3.3/asciitree-0.3.3
# !pip install --no-index --find-links=/kaggle/working/wheel_file zarr

In [3]:
import zarr
from ultralytics import YOLO
from tqdm import tqdm
import glob, os
import torch

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [5]:
DATA_KAGGLE_DIR = '../../raw'
EXP_NAME = "cv_mod_resize_ver2"
LIST_CV = [
    'TS_5_4',
    'TS_69_2',
    'TS_6_4',
    'TS_6_6',
]

In [6]:
WANDB = True
WANDB_EXP_NAME = f"{EXP_NAME}"
# EXP_NAME = "try"

if WANDB:
    # !pip install wandb
    import wandb
    import os
    from dotenv import load_dotenv
    load_dotenv()
    wandb.login(key=os.environ.get("WANDB_API_KEY"))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: trira7503 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


We use a recursive function in this notebook, and we change the settings to explore the graph deep enough.

In [7]:
import sys
sys.setrecursionlimit(10000)

In [8]:
import warnings
warnings.simplefilter('ignore')
np.warnings = warnings

# Prepare trained YOLO model

In [9]:
#add by @minfuka
assert torch.cuda.device_count() == 1

In [10]:
particle_names = ['apo-ferritin', 'beta-amylase', 'beta-galactosidase', 'ribosome', 'thyroglobulin', 'virus-like-particle']

In [11]:
p2i_dict = {
        'apo-ferritin': 0,
        'beta-amylase': 1,
        'beta-galactosidase': 2,
        'ribosome': 3,
        'thyroglobulin': 4,
        'virus-like-particle': 5
    }

i2p = {v:k for k, v in p2i_dict.items()}

In [12]:
particle_radius = {
        'apo-ferritin': 60,
        'beta-amylase': 65,
        'beta-galactosidase': 90,
        'ribosome': 150,
        'thyroglobulin': 130,
        'virus-like-particle': 135,
    }

# define Main process class
There are many variables and functions going back and forth. We can easily manage variables by defining classes.

In [13]:
class PredAggForYOLO:
    def __init__(self, first_conf=0.2, final_conf=0.3, conf_coef=0.75):
        self.first_conf = first_conf # threshold of confidence yolo
        self.final_conf = final_conf # final threshold score (not be used in version 14)
        self.conf_coef = conf_coef # if found many points, give bonus
        self.particle_confs = [0.5, 0.0, 0.2, 0.5, 0.2, 0.5] # be strict to easy labels 

    def convert_to_8bit(self, x):
        lower, upper = np.percentile(x, (0.5, 99.5))
        x = np.clip(x, lower, upper)
        x = (x - x.min()) / (x.max() - x.min() + 1e-12) * 255
        return x.round().astype("uint8")

    # depth first search.
    # aggregate the coordinates and confidence scores of connected graphs.
    def dfs(self, v):
        self.passed[v] = True
        self.conf_sum += self.pdf.iloc[v].confidence
        self.cx += self.pdf.iloc[v].x
        self.cy += self.pdf.iloc[v].y
        self.cz += self.pdf.iloc[v].z
        self.nv += 1
        for next_v in self.adjacency_list[v]:
            if (self.passed[next_v]): continue
            self.dfs(next_v)

    # main routine.
    # change by @minfuka
    # def make_predict_yolo(self, r, model):
    def make_predict_yolo(self, r, model, device_no):
        vol = zarr.open(f'{valid_dir}/static/ExperimentRuns/{r}/VoxelSpacing10.000/denoised.zarr', mode='r')
        vol = vol[0]
        vol2 = self.convert_to_8bit(vol)
        n_imgs = vol2.shape[0]
    
        df = pd.DataFrame()
    
        pts = []
        confs = []
        xs = []
        ys = []
        zs = []
        
        for i in range(n_imgs):
            # Unfortunately the image size needs to be a multiple of 32.
            tmp_img = np.zeros((630, 630))
            tmp_img[:] = vol2[i]
    
            inp_arr = np.stack([tmp_img]*3,axis=-1)
            inp_arr = cv2.resize(inp_arr, (640,640))

            # change by @minfuka
            # res = model.predict(inp_arr, save=False, imgsz=640, conf=self.first_conf, device="0", batch=1, verbose=False)
            res = model.predict(inp_arr, save=False, imgsz=640, conf=self.first_conf, device=device_no, batch=1, verbose=False)
            for j, result in enumerate(res):
                boxes = result.boxes # Boxes object for bounding box outputs    
                for k in range(len(boxes.cls)):
                    ptype = i2p[boxes.cls.cpu().numpy()[k]] # particle type
                    conf = boxes.conf.cpu().numpy()[k] # confidence score
                    # YOLO can infer (start_x, end_x, start_y, end_y)
                    xc = (boxes.xyxy[k,0] + boxes.xyxy[k,2]) / 2.0 * 10.012 * (63/64)
                    yc = (boxes.xyxy[k,1] + boxes.xyxy[k,3]) / 2.0 * 10.012 * (63/64)
                    zc = i * 10 + 5
    
                    pts.append(ptype)
                    confs.append(conf)
                    xs.append(xc.cpu().numpy())
                    ys.append(yc.cpu().numpy())
                    zs.append(zc)           
                
        df['particle_type'] = pts
        df['confidence'] = confs
        df['x'] = xs
        df['y'] = ys
        df['z'] = zs

        # df includes overall canditate of CIRCLE. 
        df = df.sort_values(['particle_type', 'z'], ascending=[True, True])
    
        agg_df = []

        # infer center of sphere each particle types
        for pidx, p in enumerate(particle_names):
            if p == 'beta-amylase':
                continue
            pdf = df[df['particle_type']==p].reset_index(drop=True)
            self.pdf = pdf
            p_rad = particle_radius[p]

            # The distance between the x and y coordinates of adjacent slices is expected to be very small.
            xy_tol = p_rad / 16.0
            xy_tol_p2 = xy_tol ** 2

            # define the graph
            self.adjacency_list = [[] for _ in range(len(pdf))]
            # which already passed in dfs
            self.passed = [False for _ in range(len(pdf))]

            # Connect two points when they are close enough
            for i in range(len(pdf)):
                x1 = pdf['x'].iloc[i]
                y1 = pdf['y'].iloc[i]
                z1 = pdf['z'].iloc[i]
                for j in range(i+1, len(pdf), 1):
                    x2 = pdf['x'].iloc[j]
                    y2 = pdf['y'].iloc[j]
                    z2 = pdf['z'].iloc[j]
                    # Can be pruned. thanks to min fuka (@minfuka)
                    if abs(z1-z2)>20:
                        break
    
                    dist_p2 = (x1-x2)**2 + (y1-y2)**2
                    if dist_p2<xy_tol_p2 and dist_p2+(z1-z2)**2 < p_rad**2 and abs(z1-z2)<=20:
                        self.adjacency_list[i].append(j)
                        self.adjacency_list[j].append(i)

            rdf = pd.DataFrame()
            cxs = []
            cys = []
            czs = []

            # Perform DFS on all points and find the center of the sphere from the average of the coordinates
            for i in range(len(pdf)):
                self.conf_sum = 0
                self.nv = 0
                self.cx = 0
                self.cy = 0
                self.cz = 0
                if not self.passed[i]:
                    self.dfs(i)

                # Different confidence for different particle types
                if self.nv>=2 and self.conf_sum / (self.nv**self.conf_coef) > self.particle_confs[pidx]:
                    cxs.append(self.cx / self.nv)
                    cys.append(self.cy / self.nv)
                    czs.append(self.cz / self.nv)

            rdf['experiment'] = [r] * len(cxs)
            rdf['particle_type'] = [p] * len(cys)
            rdf['x'] = cxs
            rdf['y'] = cys
            rdf['z'] = czs

            agg_df.append(rdf)

       
        return pd.concat(agg_df, axis=0)

In [14]:
# instance main class
agent = PredAggForYOLO(first_conf=0.15, final_conf=0.2, conf_coef=0.5) # final_conf is not used after version 14
# subs = []

In [15]:
# subs = []

In [16]:
import time
#add by @minfuka
from concurrent.futures import ProcessPoolExecutor #add by @minfuka

# main loop of inference

In [17]:
valid_dir =f'{DATA_KAGGLE_DIR}/train'
list_model_path = [
    f"../../runs/detect/{EXP_NAME}/weights/best.pt",
    f"../../runs/detect/{EXP_NAME}2/weights/best.pt",
    f"../../runs/detect/{EXP_NAME}3/weights/best.pt",
    f"../../runs/detect/{EXP_NAME}4/weights/best.pt",
]

In [18]:
#add by @minfuka
def inference(runs, model, device_no):
    subs = []
    for r in tqdm(runs, total=len(runs)):
        df = agent.make_predict_yolo(r, model, device_no)
        subs.append(df)
    
    return subs

In [19]:

# tick = time.time()
#change by @minfuka
subs = []
for r, model_path in tqdm(zip(LIST_CV, list_model_path), total=len(LIST_CV)):
    model = YOLO(model_path)
    df = agent.make_predict_yolo(r, model, "0")
    subs.append(df)
# with ProcessPoolExecutor(max_workers=2) as executor:
#     results = list(executor.map(inference, (runs1, runs2), (model, model), ("0", "1")))
# tock = time.time()

100%|██████████| 4/4 [01:32<00:00, 23.18s/it]


In [20]:
df

,experiment,particle_type,x,y,z
0,TS_6_6,apo-ferritin,5358.622314,451.889399,70.000000
1,TS_6_6,apo-ferritin,5890.453906,2164.746826,165.000000
2,TS_6_6,apo-ferritin,5072.097819,1959.123210,225.000000
3,TS_6_6,apo-ferritin,4919.313314,1095.556356,265.000000
4,TS_6_6,apo-ferritin,3114.478809,1283.814233,255.000000
...,...,...,...,...,...
30,TS_6_6,virus-like-particle,2617.775757,4580.211670,1180.000000
31,TS_6_6,virus-like-particle,432.530411,2718.590820,1110.000000
32,TS_6_6,virus-like-particle,3014.094130,6169.306586,1166.111111
33,TS_6_6,virus-like-particle,2219.445479,4138.427871,1295.000000


In [21]:
#submission = pd.concat(subs).reset_index(drop=True)
#change by @minfuka
# submission1 = pd.concat(results[1])
# if len(valid_id) == 1:
#     submission = submission1.copy()
# else:
#     submission0 = pd.concat(results[0])
#     submission = pd.concat([submission0, submission1]).reset_index(drop=True)
submission = pd.concat(subs).reset_index(drop=True)
submission.insert(0, 'id', range(len(submission)))

In [22]:
submission.to_csv(f"../../proc/sub/submission_{EXP_NAME}.csv", index=False)
submission.head()

,id,experiment,particle_type,x,y,z
0,0,TS_5_4,apo-ferritin,5882.576714,5132.180827,85.0
1,1,TS_5_4,apo-ferritin,5752.842407,5107.837402,90.0
2,2,TS_5_4,apo-ferritin,5723.539453,5004.981348,125.0
3,3,TS_5_4,apo-ferritin,5753.327148,5114.914551,120.0
4,4,TS_5_4,apo-ferritin,5301.001831,4171.303345,140.0


# Scoring

https://www.kaggle.com/code/hengck23/3d-unet-using-2d-image-encoder/notebook

In [23]:
import sys
sys.path.append('hengck')

from czii_helper import *
from dataset import *
from model2 import *
import numpy as np
from scipy.optimize import linear_sum_assignment

In [24]:
def do_one_eval(truth, predict, threshold):
    P=len(predict)
    T=len(truth)

    if P==0:
        hit=[[],[]]
        miss=np.arange(T).tolist()
        fp=[]
        metric = [P,T,len(hit[0]),len(miss),len(fp)]
        return hit, fp, miss, metric

    if T==0:
        hit=[[],[]]
        fp=np.arange(P).tolist()
        miss=[]
        metric = [P,T,len(hit[0]),len(miss),len(fp)]
        return hit, fp, miss, metric

    #---
    distance = predict.reshape(P,1,3)-truth.reshape(1,T,3)
    distance = distance**2
    distance = distance.sum(axis=2)
    distance = np.sqrt(distance)
    p_index, t_index = linear_sum_assignment(distance)

    valid = distance[p_index, t_index] <= threshold
    p_index = p_index[valid]
    t_index = t_index[valid]
    hit = [p_index.tolist(), t_index.tolist()]
    miss = np.arange(T)
    miss = miss[~np.isin(miss,t_index)].tolist()
    fp = np.arange(P)
    fp = fp[~np.isin(fp,p_index)].tolist()

    metric = [P,T,len(hit[0]),len(miss),len(fp)] #for lb metric F-beta copmutation
    return hit, fp, miss, metric


def compute_lb(submit_df, overlay_dir):
    valid_id = list(submit_df['experiment'].unique())
    print(valid_id)

    eval_df = []
    for id in valid_id:
        truth = read_one_truth(id, overlay_dir) #=f'{valid_dir}/overlay/ExperimentRuns')
        id_df = submit_df[submit_df['experiment'] == id]
        for p in PARTICLE:
            p = dotdict(p)
            print('\r', id, p.name, end='', flush=True)
            xyz_truth = truth[p.name]
            xyz_predict = id_df[id_df['particle_type'] == p.name][['x', 'y', 'z']].values
            hit, fp, miss, metric = do_one_eval(xyz_truth, xyz_predict, p.radius* 0.5)
            eval_df.append(dotdict(
                id=id, particle_type=p.name,
                P=metric[0], T=metric[1], hit=metric[2], miss=metric[3], fp=metric[4],
            ))
    print('')
    eval_df_all = pd.DataFrame(eval_df)
    gb_all = []
    lb_score_all = []
    for exp in LIST_CV:
        eval_df = eval_df_all[eval_df_all['id'] == exp]
        gb = eval_df.groupby('particle_type').agg('sum').drop(columns=['id'])
        gb.loc[:, 'precision'] = gb['hit'] / gb['P']
        gb.loc[:, 'precision'] = gb['precision'].fillna(0)
        gb.loc[:, 'recall'] = gb['hit'] / gb['T']
        gb.loc[:, 'recall'] = gb['recall'].fillna(0)
        gb.loc[:, 'f-beta4'] = 17 * gb['precision'] * gb['recall'] / (16 * gb['precision'] + gb['recall'])
        gb.loc[:, 'f-beta4'] = gb['f-beta4'].fillna(0)

        gb = gb.sort_values('particle_type').reset_index(drop=False)
        # https://www.kaggle.com/competitions/czii-cryo-et-object-identification/discussion/544895
        gb.loc[:, 'weight'] = [1, 0, 2, 1, 2, 1]
        lb_score = (gb['f-beta4'] * gb['weight']).sum() / gb['weight'].sum()
        gb_all.append(gb)
        lb_score_all.append(lb_score)
    return gb_all, lb_score_all


def score_submission(submission):
    #if 1:
    submit_df=submission.copy()
    gb_all, lb_score_all = compute_lb(submit_df, '../../raw/train/overlay/ExperimentRuns')
    for gb, lb_score in zip(gb_all, lb_score_all):
        display(gb)
        print(f'lb_score: {lb_score:.4f}')
        print('')
        print("--------------------------------")

    return lb_score_all


    #show one ----------------------------------
    # fig = plt.figure(figsize=(18, 8))

    # id = valid_id[0]
    # truth = read_one_truth(id,overlay_dir=f'{valid_dir}/overlay/ExperimentRuns')

    # submit_df = submit_df[submit_df['experiment']==id]
    # for p in PARTICLE:
    #     p = dotdict(p)
    #     xyz_truth = truth[p.name]
    #     xyz_predict = submit_df[submit_df['particle_type']==p.name][['x','y','z']].values
    #     hit, fp, miss, _ = do_one_eval(xyz_truth, xyz_predict, p.radius)
    #     print(id, p.name)
    #     print('\t num truth   :',len(xyz_truth) )
    #     print('\t num predict :',len(xyz_predict) )
    #     print('\t num hit  :',len(hit[0]) )
    #     print('\t num fp   :',len(fp) )
    #     print('\t num miss :',len(miss) )

    #     ax = fig.add_subplot(2, 3, p.label, projection='3d')
    #     if hit[0]:
    #         pt = xyz_predict[hit[0]]
    #         ax.scatter(pt[:, 0], pt[:, 1], pt[:, 2], alpha=0.5, color='r')
    #         pt = xyz_truth[hit[1]]
    #         ax.scatter(pt[:,0], pt[:,1], pt[:,2], s=80, facecolors='none', edgecolors='r')
    #     if fp:
    #         pt = xyz_predict[fp]
    #         ax.scatter(pt[:, 0], pt[:, 1], pt[:, 2], alpha=1, color='k')
    #     if miss:
    #         pt = xyz_truth[miss]
    #         ax.scatter(pt[:, 0], pt[:, 1], pt[:, 2], s=160, alpha=1, facecolors='none', edgecolors='k')

    #     ax.set_title(f'{p.name} ({p.difficulty})')

    # plt.tight_layout()
    # plt.show()
    
    # #--- 
    # zz=0

In [25]:
lb_score_all = score_submission(submission)

['TS_5_4', 'TS_69_2', 'TS_6_4', 'TS_6_6']
 TS_6_6 virus-like-particlee


,particle_type,P,T,hit,miss,fp,precision,recall,f-beta4,weight
0,apo-ferritin,63,46,36,10,27,0.571429,0.782609,0.765957,1
1,beta-amylase,0,10,0,10,0,0.000000,0.000000,0.000000,0
2,beta-galactosidase,24,12,7,5,17,0.291667,0.583333,0.550926,2
3,ribosome,57,31,24,7,33,0.421053,0.774194,0.737794,1
4,thyroglobulin,87,30,18,12,69,0.206897,0.600000,0.539683,2
5,virus-like-particle,17,11,11,0,6,0.647059,1.000000,0.968912,1


lb_score: 0.6648

--------------------------------


,particle_type,P,T,hit,miss,fp,precision,recall,f-beta4,weight
0,apo-ferritin,70,35,35,0,35,0.500000,1.000000,0.944444,1
1,beta-amylase,0,12,0,12,0,0.000000,0.000000,0.000000,0
2,beta-galactosidase,48,16,10,6,38,0.208333,0.625000,0.559211,2
3,ribosome,67,37,33,4,34,0.492537,0.891892,0.851290,1
4,thyroglobulin,127,34,27,7,100,0.212598,0.794118,0.684054,2
5,virus-like-particle,21,9,9,0,12,0.428571,1.000000,0.927273,1


lb_score: 0.7442

--------------------------------


,particle_type,P,T,hit,miss,fp,precision,recall,f-beta4,weight
0,apo-ferritin,120,58,56,2,64,0.466667,0.965517,0.908397,1
1,beta-amylase,0,9,0,9,0,0.000000,0.000000,0.000000,0
2,beta-galactosidase,37,12,8,4,29,0.216216,0.666667,0.593886,2
3,ribosome,171,74,69,5,102,0.403509,0.932432,0.865683,1
4,thyroglobulin,171,30,26,4,145,0.152047,0.866667,0.678955,2
5,virus-like-particle,22,10,8,2,14,0.363636,0.800000,0.747253,1


lb_score: 0.7239

--------------------------------


,particle_type,P,T,hit,miss,fp,precision,recall,f-beta4,weight
0,apo-ferritin,71,41,40,1,31,0.563380,0.975610,0.935351,1
1,beta-amylase,0,14,0,14,0,0.000000,0.000000,0.000000,0
2,beta-galactosidase,43,11,6,5,37,0.139535,0.545455,0.465753,2
3,ribosome,31,23,19,4,12,0.612903,0.826087,0.809524,1
4,thyroglobulin,218,35,29,6,189,0.133028,0.828571,0.633676,2
5,virus-like-particle,35,19,19,0,16,0.542857,1.000000,0.952802,1


lb_score: 0.6995

--------------------------------


In [26]:
# wandbの初期化
if WANDB:
    wandb_config = {
        # ... 既存の設定 ...
        # "epochs": CONFIG['epochs'],
        # "learning_rate": CONFIG['learning_rate'],
        # "min_lr": CONFIG["min_lr"],
        # "weight_decay": CONFIG["weight_decay"],
        # "mixup_alpha": CONFIG["mixup_alpha"],
        # "mixup_epochs": CONFIG["mixup_epochs"],  # 新しく追加
    }
    wandb.init(project="CZII", name=WANDB_EXP_NAME, config=wandb_config)

for exp, score in zip(LIST_CV, lb_score_all):
    print(f'lb_score: {score:.4f}')
    if WANDB:
        wandb.log({f"lb_score_{exp}": score})
print(f'mean: {np.mean(lb_score_all):.4f}')
if WANDB:
    wandb.log({"mean_lb_score": np.mean(lb_score_all)})
    wandb.finish()



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


lb_score: 0.6648
lb_score: 0.7442
lb_score: 0.7239
lb_score: 0.6995
mean: 0.7081


lb_score_TS_5_4,▁
lb_score_TS_69_2,▁
lb_score_TS_6_4,▁
lb_score_TS_6_6,▁
mean_lb_score,▁
lb_score_TS_5_4,0.66484
lb_score_TS_69_2,0.74422
lb_score_TS_6_4,0.72386
lb_score_TS_6_6,0.69951
mean_lb_score,0.70811
